# 03 - Grid / Block Model
Genera la grilla de bloques desde config o extents.

In [1]:
import os, sys, json

PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
if os.path.basename(os.getcwd()) == "notebooks":
    os.chdir(PROJECT_ROOT)
else:
    PROJECT_ROOT = os.getcwd()

sys.path.insert(0, PROJECT_ROOT)

In [2]:
from src.io_utils import read_csv_robust, standardize_columns, coerce_numeric
from src.grid import grid_from_extents, make_grid_dataframe, export_grid_to_csv

with open("config/project.json", "r", encoding="utf-8") as f:
    cfg = json.load(f)

if not os.path.exists(cfg["data_csv_path"]):
    from src.make_demo_data import make_demo_csv
    make_demo_csv(cfg["data_csv_path"])

df = read_csv_robust(cfg["data_csv_path"])

mapping = {
    "x": cfg["columns"].get("x"),
    "y": cfg["columns"].get("y"),
    "z": cfg["columns"].get("z"),
    "var": cfg["columns"].get("variable_objetivo"),
}

df = standardize_columns(df, mapping)
coerce_numeric(df, ["x", "y", "z", "var"])
df.dropna(subset=["x", "y", "var"], inplace=True)

grid_cfg = cfg.get("grid", {})
if grid_cfg.get("auto_from_data", True):
    grid_spec = grid_from_extents(
        df,
        "x",
        "y",
        "z",
        float(grid_cfg.get("dx", 25.0)),
        float(grid_cfg.get("dy", 25.0)),
        float(grid_cfg.get("dz", 5.0)),
        pad=float(grid_cfg.get("pad", 0.0)),
    )
else:
    grid_spec = {
        "nx": grid_cfg["nx"],
        "ny": grid_cfg["ny"],
        "nz": grid_cfg.get("nz", 1),
        "xmin": grid_cfg["xmin"],
        "ymin": grid_cfg["ymin"],
        "zmin": grid_cfg.get("zmin", 0.0),
        "dx": grid_cfg["dx"],
        "dy": grid_cfg["dy"],
        "dz": grid_cfg.get("dz", 1.0),
    }

grid_df = make_grid_dataframe(grid_spec)
os.makedirs("outputs/tables", exist_ok=True)
export_grid_to_csv(grid_df, "outputs/tables/grid.csv")

print(grid_df.head())

             x             y            z
0  2363723.332  6.485698e+06  3107.389936
1  2363723.332  6.485698e+06  3112.389936
2  2363723.332  6.485698e+06  3117.389936
3  2363723.332  6.485698e+06  3122.389936
4  2363723.332  6.485698e+06  3127.389936
